# Utilities for the models of this thesis
> comment

In [1]:
# | default_exp models.util

In [2]:
# | export
# libraries
import matplotlib.pyplot as plt
from isssm.typing import PGSSM
from matplotlib.colors import Normalize
import matplotlib.cm as cm
import jax.numpy as jnp
from jaxtyping import Float, Array
import fastcore.test as fct

## Visualization

In [3]:
# | export


def __zero_to_nan(arr, eps=1e-10):
    return jnp.where(jnp.abs(arr) < eps, jnp.nan, arr)


def visualize_pgssm(pgssm: PGSSM):
    fig, axes = plt.subplots(nrows=1, ncols=3)
    cmap = cm.get_cmap("viridis")

    A, B, D, Sigma = pgssm.A[0], pgssm.B[0], pgssm.D[0], pgssm.Sigma[0]
    max = jnp.max(jnp.array([A.max(), B.max(), D.max()]))
    min = jnp.min(jnp.array([A.min(), B.min(), D.min()]))

    normalizer = Normalize(min, max)
    im = cm.ScalarMappable(norm=normalizer)
    axes[0].imshow(__zero_to_nan(A), cmap=cmap, norm=normalizer)
    axes[0].set_title("A")
    axes[1].imshow(__zero_to_nan(B), cmap=cmap, norm=normalizer)
    axes[1].set_title("B")
    axes[2].imshow(__zero_to_nan(D), cmap=cmap, norm=normalizer)
    axes[2].set_title("D")

    fig.colorbar(im, ax=axes.ravel().tolist())
    plt.show()

    plt.imshow(__zero_to_nan(Sigma))
    plt.colorbar()
    plt.show()

## Computation

For $p \in \mathbf R^{k}_{>0}$ with $\sum_{i = 1}^k p_{i} = 1$, let $\log q_i = \log \frac{p_{i}}{p_{k}}$ for $i = 1, \dots, k -1$. Then 
$$
    p_{k} = \frac{1}{1 + \sum_{i = 1}^{k-1}q_{i}},
$$
so 
$$
    p_{i} = q_{i} p_{k} = \frac{q_{i}}{1 + \sum_{i = 1}^{k-1}q_{i}}.

$$



In [4]:
# | export
def to_log_probs(log_ratios: Float[Array, "k-1"]) -> Float[Array, "k"]:
    exp_q = jnp.exp(log_ratios)
    p_n_delay = 1 / (1 + exp_q.sum(axis=-1, keepdims=True))
    log_p = jnp.log(jnp.concatenate([exp_q * p_n_delay, p_n_delay], axis=-1))
    return log_p

In [5]:
# | hide
fct.test_close(to_log_probs(jnp.zeros(4)), jnp.log(jnp.ones(5) / 5))

## Monkey-Patching

In [6]:
# | export
import isssm.importance_sampling


def patch_la():
    isssm.importance_sampling.log_weights = print

In [7]:
patch_la()
from isssm.importance_sampling import log_weights

log_weights("10")

10


### Full dependencies

In [ ]:
#| export
from isssm.util import MVN_degenerate as MVN

def log_weights_t_full_deps(
    s_t: Float[Array, "p"],  # signal
    y_t: Float[Array, "p"],  # observation
    xi_t: Float[Array, "p"],  # parameters
    dist,  # observation distribution
    z_t: Float[Array, "p"],  # synthetic observation
    Omega_t: Float[Array, "p p"],  # synthetic observation covariance, assumed diagonal
) -> Float:  # single log weight
    """Log weight for a single time point."""
    p_ys = dist(s_t, xi_t).log_prob(y_t).sum()

    # omega_t = jnp.sqrt(jnp.diag(Omega_t))
    # g_zs = MVN_diag(s_t, omega_t).log_prob(z_t).sum()
    g_zs = jnp.nansum(MVN(s_t, Omega_t).log_prob(z_t))

    return p_ys - g_zs


def log_weights_full_deps(
    s: Float[Array, "n+1 p"],  # signals
    y: Float[Array, "n+1 p"],  # observations
    dist,  # observation distribution
    xi: Float[Array, "n+1 p"],  # observation parameters
    z: Float[Array, "n+1 p"],  # synthetic observations
    Omega: Float[Array, "n+1 p p"],  # synthetic observation covariances:
) -> Float:  # log weights
    """Log weights for all time points"""
    p_ys = dist(s, xi).log_prob(y).sum()

    # avoid triangular solve problems
    # omega = jnp.sqrt(vmap(jnp.diag)(Omega))
    # g_zs = MVN_diag(s, omega).log_prob(z).sum()
    g_zs = jnp.nansum(MVN(s, Omega).log_prob(z))

    return p_ys - g_zs

def patch_log_weights_full_deps():
    isssm.importance_sampling.log_weights = log_weights_full_deps
    isssm.importance_sampling.log_weights_t = log_weights_t_full_deps


In [ ]:
# | export
# LA monkey patch
from isssm.kalman import kalman, smoothed_signals
from isssm.typing import GLSSM, GLSSMProposal, ConvergenceInformation
from isssm.laplace_approximation import default_link, vvmap, vdiag
from jax.scipy.optimize import minimize
from functools import partial
from isssm.util import converged
from jax import jacfwd, hessian, jacrev
from jax.lax import while_loop
import isssm.laplace_approximation


def _initial_guess_full_deps(xi_t, y_t, dist, link=default_link):
    result = minimize(
        lambda s_ti: -dist(s_ti, xi_t).log_prob(y_t).sum(),
        jnp.concatenate((jnp.atleast_1d(link(y_t.sum())), jnp.zeros(3))),
        method="BFGS",
    )
    return jnp.squeeze(result.x)


def laplace_approximation_full_deps(
    y: Float[Array, "n+1 p"],  # observation
    model: PGSSM,
    n_iter: int,  # number of iterations
    log_lik=None,  # log likelihood function
    d_log_lik=None,  # derivative of log likelihood function
    dd_log_lik=None,  # second derivative of log likelihood function
    eps: Float = 1e-5,  # precision of iterations
    link=default_link,  # default link to use in initial guess
) -> tuple[GLSSMProposal, ConvergenceInformation]:
    u, A, D, Sigma0, Sigma, v, B, dist, xi = model
    np1, p, m = B.shape

    # s_init = vmap(partial(_initial_guess_full_deps, dist=dist, link=link))(xi, y)
    # missing values have zero obs. -> 0.
    s_init = jnp.hstack(
        (jnp.log(y.sum(axis=1, keepdims=True) + 1.0), jnp.zeros((np1, 3)))
    )

    def default_log_lik(
        s_t: Float[Array, "p"], xi_t: Float[Array, "p"], y_t: Float[Array, "p'"]
    ):
        return dist(s_t, xi_t).log_prob(y_t).sum()

    if log_lik is None:
        log_lik = default_log_lik

    if d_log_lik is None:
        d_log_lik = jacfwd(log_lik, argnums=0)
    if dd_log_lik is None:
        dd_log_lik = jacrev(d_log_lik, argnums=0)

    vd_log_lik = jit(vmap(d_log_lik))
    vdd_log_lik = jit(vmap(dd_log_lik))

    def _break(val):
        _, i, z, Omega, z_old, Omega_old = val

        z_converged = jnp.logical_and(converged(z, z_old, eps), i > 0)
        Omega_converged = jnp.logical_and(converged(Omega, Omega_old, eps), i > 0)
        all_converged = jnp.logical_and(z_converged, Omega_converged)

        iteration_limit_reached = i >= n_iter

        return jnp.logical_or(all_converged, iteration_limit_reached)

    def _iteration(val):
        s, i, z_old, Omega_old, _, _ = val

        grad = vd_log_lik(s, xi, y)
        Gamma = -vdd_log_lik(s, xi, y)
        # pinv for missing values: if Gamma is 0, we want Omega to be 0 as well
        # requries exact derivatives, not numerical ones!
        Omega = jnp.linalg.pinv(Gamma, hermitian=True)

        # z = s + jnp.linalg.solve(Gamma, grad[..., None])[..., 0]
        z = s + (Omega @ grad[..., None])[..., 0]
        approx_glssm = GLSSM(u, A, D, Sigma0, Sigma, v, B, Omega)

        filtered = kalman(z, approx_glssm)
        s_new = smoothed_signals(filtered, z, approx_glssm)

        return s_new, i + 1, z, Omega, z_old, Omega_old

    empty_z = jnp.empty_like(s_init)
    empty_Omega = jnp.empty((np1, p, p))
    init = (s_init, 0, empty_z, empty_Omega, empty_z, empty_Omega)

    _keep_going = lambda *args: jnp.logical_not(_break(*args))
    _, n_iters, z, Omega, z_old, Omega_old = while_loop(_keep_going, _iteration, init)

    final_proposal = GLSSMProposal(u, A, D, Sigma0, Sigma, v, B, Omega, z)
    delta_z = jnp.max(jnp.abs(z - z_old))
    delta_Omega = jnp.max(jnp.abs(Omega - Omega_old))
    information = ConvergenceInformation(
        converged=jnp.logical_and(
            converged(z, z_old, eps), converged(Omega, Omega_old, eps)
        ),
        n_iter=n_iters,
        delta=jnp.max(jnp.array([delta_z, delta_Omega])),
    )
    return final_proposal, information


def patch_la_full_deps():
    isssm.laplace_approximation._initial_guess = _initial_guess_full_deps
    isssm.laplace_approximation.laplace_approximation = laplace_approximation_full_deps

In [ ]:
# | export
# MEIS monkey patch
from jax import jit, vmap
import jax.random as jrn
from jaxtyping import PRNGKeyArray
from isssm.importance_sampling import normalize_weights, log_weights_t
from isssm.kalman import simulation_smoother
import isssm.modified_efficient_importance_sampling


def triu(entries: Float[Array, " q"], p: int):
    (q,) = entries.shape

    if 2 * q != p * (p + 1):
        raise ValueError(
            f"entries does not have the correct shape, expected {int(p * (p - 1) /2)} entries in the second dimension but got {q}"
        )

    mat = jnp.zeros((p, p))
    mat = mat.at[jnp.triu_indices(p)].set(entries)

    return mat


def tril(entries: Float[Array, " q"], p: int):
    (q,) = entries.shape

    if 2 * q != p * (p + 1):
        raise ValueError(
            f"entries does not have the correct shape, expected {int(p * (p - 1) /2)} entries in the second dimension but got {q}"
        )

    mat = jnp.zeros((p, p))
    mat = mat.at[jnp.tril_indices(p)].set(entries)

    return mat


@jit
def optimal_parameters(
    signal: Float[Array, "N p"], weights: Float[Array, "N"], log_p: Float[Array, "N"]
):
    N, p = signal.shape
    ones = jnp.ones_like(weights)[:, None]

    second_moment = vmap(jnp.outer)(signal, signal)
    # double tranpose, because [:, jnp.tril_indices(p)] is not supported (returns (N, 2, p(p-1)/2, 4) instead of (N, p(p-1)/2)
    lower_tri_second_moment = second_moment.transpose((1, 2, 0))[jnp.tril_indices(p)].T

    # w_inner_prod = lambda a, b: jnp.einsum("i,ij,ik->jk", weights, a, b)
    # X_T_W_X = jnp.block(
    #    [
    #        [
    #            w_inner_prod(ones, ones),
    #            w_inner_prod(ones, signal),
    #            w_inner_prod(ones, -0.5 * lower_tri_second_moment),
    #        ],
    #        [
    #            w_inner_prod(signal, ones),
    #            w_inner_prod(signal, signal),
    #            w_inner_prod(signal, -0.5 * lower_tri_second_moment),
    #        ],
    #        [
    #            w_inner_prod(-0.5 * lower_tri_second_moment, ones),
    #            w_inner_prod(-0.5 * lower_tri_second_moment, signal),
    #            w_inner_prod(
    #                -0.5 * lower_tri_second_moment, -0.5 * lower_tri_second_moment
    #            ),
    #        ],
    #    ]
    # )
    # X_T_W_y = jnp.concatenate(
    #    [
    #        w_inner_prod(ones, log_p[:, None]),
    #        w_inner_prod(signal, log_p[:, None]),
    #        w_inner_prod(-0.5 * lower_tri_second_moment, log_p[:, None]),
    #    ]
    # )

    design = jnp.hstack([ones, signal, -0.5 * lower_tri_second_moment])
    lhs = log_p

    sqrt_weights = jnp.sqrt(weights)

    design_W = design * sqrt_weights[:, None]
    lhs_W = lhs * sqrt_weights

    # if lhs_W is 0, this returns 0 even if design matrix does not have full rank (numpy promises solution with least l2-norm)
    beta, *_ = jnp.linalg.lstsq(design_W, lhs_W)
    return beta
    # beta = jnp.linalg.solve(X_T_W_X, X_T_W_y[:, 0])
    # return beta


def MEIS_full_deps(
    y: Float[Array, "n+1 p"],  # observations
    model: PGSSM,  # model
    z_init: Float[Array, "n+1 p"],  # initial z estimate
    Omega_init: Float[Array, "n+1 p p"],  # initial Omega estimate
    n_iter: int,  # number of iterations
    N: int,  # number of samples
    key: PRNGKeyArray,  # random key
    eps: Float = 1e-5,  # convergence threshold
):
    z, Omega = z_init, Omega_init

    np1, p, m = model.B.shape

    key, crn_key = jrn.split(key)

    v_norm_w = vmap(normalize_weights)
    dist = model.dist
    lw_t = vmap(
        vmap(lambda s, y, xi, z, Omega: log_weights_t(s, y, xi, dist, z, Omega)),
        (0, None, None, None, None),
    )

    def _break(val):
        i, z, Omega, z_old, Omega_old = val

        # in first iteration we don't have old values, converged is True for NaNs
        z_converged = jnp.logical_and(converged(z, z_old, eps), i > 0)
        Omega_converged = jnp.logical_and(converged(Omega, Omega_old, eps), i > 0)
        iteration_limit_reached = i >= n_iter

        return jnp.logical_or(
            jnp.logical_and(z_converged, Omega_converged), iteration_limit_reached
        )

    def _iteration(val):
        i, z, Omega, _, _ = val
        glssm_approx = GLSSM(
            model.u,
            model.A,
            model.D,
            model.Sigma0,
            model.Sigma,
            model.v,
            model.B,
            Omega,
        )
        sim_signal = simulation_smoother(glssm_approx, z, N, crn_key)

        log_weights = lw_t(sim_signal, y, model.xi, z, Omega)
        log_p = dist(sim_signal, model.xi).log_prob(y).sum(axis=-1)
        wls_estimate = vmap(optimal_parameters, (1, 1, 1), 0)(
            sim_signal, v_norm_w(log_weights), log_p
        )

        a = wls_estimate[:, 0]
        b = wls_estimate[:, 1 : (p + 1)]
        c = wls_estimate[:, (p + 1) :]

        Omega_inv_new = vmap(tril, (0, None))(c, p)
        Omega_inv_new = 1 / 2 * (Omega_inv_new + Omega_inv_new.transpose((0, 2, 1)))
        # pinv for missing cases
        Omega_new = jnp.linalg.pinv(Omega_inv_new, hermitian=True)
        z_new = (Omega_new @ b[..., None])[..., 0]

        return i + 1, z_new, Omega_new, z, Omega

    _keep_going = lambda *args: jnp.logical_not(_break(*args))

    init = (0, z_init, Omega_init, jnp.empty_like(z_init), jnp.empty_like(Omega_init))
    n_iters, z, Omega, z_old, Omega_old = while_loop(_keep_going, _iteration, init)

    proposal = GLSSMProposal(
        model.u, model.A, model.D, model.Sigma0, model.Sigma, model.v, model.B, Omega, z
    )

    delta_z = jnp.max(jnp.abs(z - z_old))
    delta_Omega = jnp.max(jnp.abs(Omega - Omega_old))
    information = ConvergenceInformation(
        converged=jnp.logical_and(
            converged(z, z_old, eps), converged(Omega, Omega_old, eps)
        ),
        n_iter=n_iters,
        delta=jnp.max(jnp.array([delta_z, delta_Omega])),
    )

    return proposal, information


def patch_meis_full_deps():
    isssm.modified_efficient_importance_sampling.modified_efficient_importance_sampling = MEIS_full_deps

In [ ]:
#| export
# monkey patch gnll to handle nans
vmm = vmap(jnp.matmul, (0,0))
from isssm.util import MVN_degenerate as MVN
import isssm.estimation
@jit
def gnll_full_deps(
    y: Float[Array, "n+1 p"],  # observations $y_t$
    x_pred: Float[Array, "n+1 m"],  # predicted states $\hat X_{t+1\bar t}$
    Xi_pred: Float[Array, "n+1 m m"],  # predicted state covariances $\Xi_{t+1\bar t}$
    B: Float[Array, "n+1 p m"],  # state observation matrices $B_{t}$
    Omega: Float[Array, "n+1 p p"],  # observation covariances $\Omega_{t}$
) -> Float:  # gaussian negative log-likelihood
    """Gaussian negative log-likelihood"""
    y_pred = vmm(B, x_pred)
    Psi_pred = vmm(vmm(B, Xi_pred), jnp.transpose(B, (0, 2, 1))) + Omega

    return jnp.nansum(-MVN(y_pred, Psi_pred).log_prob(y))

def patch_gnll_full_deps():
    isssm.estimation.gnll = gnll_full_deps

In [ ]:
def patch_full_deps():
    patch_la_full_deps()
    patch_meis_full_deps()
    patch_log_weights_full_deps()
    patch_gnll_full_deps()

In [8]:
# | hide
import nbdev

nbdev.nbdev_export()

/opt/homebrew/Caskroom/miniconda/base/envs/research/lib/python3.10/site-packages/nbdev/export.py:73: UserWarning: Notebook '/Users/stefan/workspace/work/phd/thesis/nbs/4 Models/4.3 Nowcasting hospitalizations/10_model.ipynb' uses `#|export` without `#|default_exp` cell.
Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.
See https://nbdev.fast.ai/getting_started.html for more information.
  warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
